 # Homework 5 

 ### 2 . CIFAR10 - Classification 

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

Feature structure: 
id: Text(dtype=object)
image: Image(shape=(32,32,3), dtype=uint8)
label: ClassLabel(dtype=int64, num_classes=10)

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import image
import numpy as np


In [ ]:
(train_ds, test_ds), ds_info = tfds.load('cifar10', split=['train', 'test'], as_supervised=True, with_info = True)
#print(ds_info)

In [ ]:
def prepare_cifar10_data(cifar, batchsize = 32):
  '''
  This function prepares the dataset for usage in a learning model

  Args:
  cifar --- The cifar-10 dataset 
  batchsize --- The batch size (default:32)

  '''
  #convert data from uint8 to float32 
  cifar = cifar.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
  cifar = cifar.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot targets
  cifar = cifar.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  #cache this progress in memory
  cifar = cifar.cache()
  #shuffle, batch, prefetch
  cifar = cifar.shuffle(1000)
  cifar = cifar.batch(32)
  cifar= cifar.prefetch(tf.data.AUTOTUNE)
  #return preprocessed dataset
  return cifar


train_dataset = train_ds.apply(prepare_cifar10_data)
test_dataset = test_ds.apply(prepare_cifar10_data)

In [ ]:
tfds.visualization.show_examples(test_ds, ds_info)

In [ ]:
class ConvModel(tf.keras.model):
    def __init__(self, layers, optimizer):
        super().__init__()

        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = optimizer

        self.layers = layers
        self.out = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

        # instantiate metric objects to keep track of the training/test loss and accuracy
        self.loss_metric = tf.keras.metrics.Mean(name="loss") 
        self.accuracy_metric = tf.keras.metrics.CategoricalAccuracy(name="accuracy")